# SemVer: Artifact Versioning Policy (AVP)

---

Owner: Vadim Rudakov, lefthand67@gmail.com  
Version: 0.1.1  
Creation Date: 2025-10-06    
Modification Date: 2026-01-11

---

**Standardized Versioning for High-Integrity Systems**

This handbook defines the mandatory protocol for versioning any system artifact—including code, configuration, prompts, and data schemas. The goal is to ensure **Environment Parity** and **Failure Isolation** by treating every component as an immutable, traceable entity.

:::{seealso}
> [Semantic Versioning 2.0.0](https://semver.org/)
:::

## 1. The Versioning Logic (SemVer)

All artifacts must follow the `<MAJOR>.<MINOR>.<PATCH>` format.

| Segment | Significance | Criteria for Increment | Compatibility |
| --- | --- | --- | --- |
| **MAJOR**<br>`1.0.0` → `2.0.0` | **Breaking** | Changes to mandatory keys, output schemas, or logic that breaks downstream consumers. | Incompatible |
| **MINOR**<br>`1.0.0` → `1.1.0` | **Feature** | New optional fields, improved logic, or non-breaking extensions. | Backward-Compatible |
| **PATCH**<br>`1.1.0` → `1.1.1` | **Fix** | Typo corrections, internal refactoring, or parameter tuning (e.g., temperature, timeouts). | Backward-Compatible |

## 2. Mandatory Metadata Schema

Every artifact file (or its associated sidecar) **MUST** contain a metadata block to satisfy [ISO 29148: Traceability].

```json
{
  "metadata": {
    "id": "unique_artifact_identifier",
    "version": "X.Y.Z",
    "description": "Functional purpose of this version.",
    "provenance": {
      "author": "user@example.com",
      "timestamp": "YYYY-MM-DDTHH:MM:SSZ",
      "commit_sha": "git_hash_here"
    },
    "integrity": {
      "checksum": "sha256_hash",
      "validation_schema": "v1"
    }
  }
}
```

## 3. Operational Requirements

### 3.1. Immutability Law

* **No In-Place Edits**: Once a version is tagged and merged to the main branch, it is finalized. Any change, no matter how small, requires a version increment.
* **Unique Identity**: A single version number must point to exactly one state of the artifact across all environments.

:::{tip} The "State Drift" Problem
:class: dropdown
Without Unique Identity, a system suffers from **Mutable Artifacts**.

* **The Scenario:** You fix a typo in a prompt or config file but keep the version as `1.2.0` because it was "just a small change."
* **The Result:** Your local machine has "New 1.2.0," while the Production server is still running "Old 1.2.0."
* **The Technical Debt:** You can no longer debug the production environment because your local "source of truth" no longer matches the live state.
:::

### 3.2. Change Documentation [SWEBOK: Quality-2.1]

Every increment requires a entry in a `CHANGELOG.md` (if it exists for this documentation type) or the `metadata.description` field detailing:

1. **WHAT**: The specific technical change.
2. **WHY**: The architectural or business justification.
3. **IMPACT**: Potential effects on downstream systems (Critical for MAJOR versions).

### 3.3. Validation & CI/CD Integration

* **Schema Enforcement**: Use tools (e.g., `pydantic` or `json-schema`) to validate metadata before every commit.
* **Check vs. Fix**: Automation should check if the version was incremented; it should never "guess" the version for the user.

## 4. Rollback and Recovery

* **State Recovery**: Systems must be able to reference previous versions (e.g., `artifact_v1.2.0`) instantly if the current version triggers a regression.
* **Audit Trail**: The Git history acts as the legal record of truth for all version transitions.

## 5. Viability & Technical Debt

* **Classification**: **Production-Ready** (WRC 0.96).
* **Hidden Debt**: Failing to update dependencies when bumping an artifact version. Always audit the "Smallest Viable Architecture" (SVA) before a MAJOR release.